## 读取示例

### 获取多日的风光负荷

In [1]:
import pandas as pd
from pytsingrocbytelinker import PostgreLinker, DbType, WriteData, DBInfo

def read_pred_pv_province(dayplus, dbtype, source, date_begin, date_end) -> None:
    byte_linker = PostgreLinker()
    if dbtype == 'test':
        byte_linker.connect(DbType.test)
    elif dbtype == 'prod':
        byte_linker.connect(DbType.prod)
    else:
        raise ValueError("dbtype must be 'test' or 'prod'")
    db_info = DBInfo()
    db_info.read_from_data(
        table_name="power_pred_province",
        key="pred_pv",
        tags={ "source": source, "day_plus": dayplus},
        daypoints=96,
        version="v1.0.0",
        column_name="pred",
        start_time=date_begin,
        end_time=date_end,
        region="cn-shanxi",
        value_tag="value",
    )
    data = byte_linker.query(db_info)
    byte_linker.disconnect()
    data['time'] = data['time'].dt.tz_localize(None)
    return data


def read_pred_wind_province(dayplus, dbtype, source, date_begin, date_end) -> None:
    byte_linker = PostgreLinker()
    if dbtype == 'test':
        byte_linker.connect(DbType.test)
    elif dbtype == 'prod':
        byte_linker.connect(DbType.prod)
    else:
        raise ValueError("dbtype must be 'test' or 'prod'")
    db_info = DBInfo()
    db_info.read_from_data(
        table_name="power_pred_province",
        key="pred_wind",
        tags={ "source": source, "day_plus": dayplus},
        daypoints=96,
        version="v1.0.0",
        column_name="pred",
        start_time= date_begin,   #  "2025-01-01 00:00:00"
        end_time=date_end,  #"2025-05-10 23:45:00",
        region="cn-shanxi",
        value_tag="value",
    )
    data = byte_linker.query(db_info)
    byte_linker.disconnect()
    data['time'] = data['time'].dt.tz_localize(None)
    return data

def read_pred_load_province(dayplus, dbtype, source, date_begin, date_end) -> None:
    byte_linker = PostgreLinker()
    if dbtype == 'test':
        byte_linker.connect(DbType.test)
    elif dbtype == 'prod':
        byte_linker.connect(DbType.prod)
    else:
        raise ValueError("dbtype must be 'test' or 'prod'")
    db_info = DBInfo()
    db_info.read_from_data(
        table_name="load_pred",
        key="pred_load",
        tags={ "source": source, "dayplus": dayplus},
        daypoints=96,
        version="v1.0.0",
        column_name="pred",
        start_time= date_begin,   #  "2025-01-01 00:00:00"
        end_time=date_end,  #"2025-05-10 23:45:00",
        region="cn-shanxi",
        value_tag="value",
    )
    data = byte_linker.query(db_info)
    byte_linker.disconnect()
    data['time'] = data['time'].dt.tz_localize(None)
    return data

wind \
V0:清鹏-lgbm-V1.0.0-zhw-gfs \
V1:清鹏-lgbm-V1.0.0-zhw-hybrid \
V2:清鹏-lgbm-V1.0.0-zhw-ecmwfifs025 

pv \
V0:清鹏 \
V1:清鹏-lgbm-V1.0.0-zxh-反推装机容量 \
V2:清鹏-lgbm-V3.0.0-zxh-预测不限电发电量 \
V3:清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源ecmwfifs025 \
V4:清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源gfsglobal

load \
V0:清鹏

#### 风

In [2]:
dbtype = "test"
date_begin = "2025-01-01 00:00:00"

from datetime import datetime, timedelta
import pytz

beijing = pytz.timezone("Asia/Shanghai")
today_bj = datetime.now(beijing).date()
date_end = (today_bj + timedelta(days=10)).strftime("%Y-%m-%d") + " 23:45:00"


wind_dict = {
    "gfs":"清鹏-lgbm-V1.0.0-zhw-gfs",
    "ifs":"清鹏-lgbm-V1.0.0-zhw-ecmwfifs025",
    #"aifs":"清鹏-lgbm-V1.0.0-zhw-ecmwfaifs025"
}

for label,tag in wind_dict.items():
    for dayplus in range(1,6):
        try:
            wind_pred = read_pred_wind_province(dayplus, dbtype, tag, date_begin, date_end)
            wind_pred.to_parquet(f"data/processed/wind/dayplus_{dayplus}_{label}.parquet", engine='fastparquet')
        except Exception as e:
            print(f"label: {label}, tag: {tag}, dayplus: {dayplus} not found, error: {e}")
            continue


2025-08-29 01:30:25,283 - bytelinker - INFO - PostgreCore init


2025-08-29 01:30:25,558 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:26,225 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:26,265 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:26,302 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:26,483 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:27,064 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:27,102 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:27,108 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:27,291 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:27,419 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:27,421 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:27,595 - bytelinker - INFO - PostgreCore connected to test


label: gfs, tag: 清鹏-lgbm-V1.0.0-zhw-gfs, dayplus: 3 not found, error: list index out of range


2025-08-29 01:30:28,311 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:28,381 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:28,389 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:28,566 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:29,347 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:29,419 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:29,428 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:29,615 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:29,719 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:29,722 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:29,725 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:29,897 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:30,002 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:30,005 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 

#### 光

In [3]:
dbtype = "test"
date_begin = "2025-01-01 00:00:00"

from datetime import datetime, timedelta
import pytz

beijing = pytz.timezone("Asia/Shanghai")
today_bj = datetime.now(beijing).date()
date_end = (today_bj + timedelta(days=10)).strftime("%Y-%m-%d") + " 23:45:00"

pv_dict = {
    "gfs":"清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源gfsglobal",
    "ifs":"清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源ecmwfifs025",
    #"aifs":"清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源ecmwfaifs025"
}

for label,tag in pv_dict.items():
    for dayplus in range(1,6):
        try:
            pv_pred = read_pred_pv_province(dayplus, dbtype, tag, date_begin, date_end)
            pv_pred.to_parquet(f"data/processed/pv/dayplus_{dayplus}_{label}.parquet", engine='fastparquet')
        except Exception as e:
            print(f"label: {label}, tag: {tag}, dayplus: {dayplus} not found, error: {e}")
            continue


2025-08-29 01:30:31,370 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:31,549 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:32,123 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:32,162 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:32,168 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:32,349 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:32,904 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:32,942 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:32,948 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:33,437 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:34,071 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:34,113 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:34,119 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:34,305 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:3

#### 负荷

In [4]:

dbtype = "test"
date_begin = "2025-01-01 00:00:00"

from datetime import datetime, timedelta
import pytz

beijing = pytz.timezone("Asia/Shanghai")
today_bj = datetime.now(beijing).date()
date_end = (today_bj + timedelta(days=10)).strftime("%Y-%m-%d") + " 23:45:00"


load_dict = {
    "gfs":"清鹏-lasso_lgbm_mlp-gfs-V1.0.0-xzq",
    "ifs":"清鹏-lasso_lgbm_mlp-ec-V1.0.0-xzq"
}

for label,tag in load_dict.items():
    for dayplus in range(1,6):
        try:
            load_pred = read_pred_load_province(dayplus, dbtype, tag, date_begin, date_end)
            load_pred.to_parquet(f"data/processed/load/dayplus_{dayplus}_{label}.parquet", engine='fastparquet')
        except Exception as e:
            print(f"label: {label}, tag: {tag}, dayplus: {dayplus} not found, error: {e}")
            continue

2025-08-29 01:30:40,148 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:40,340 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:40,482 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:40,484 - bytelinker - INFO - PostgreCore init


label: gfs, tag: 清鹏-lasso_lgbm_mlp-gfs-V1.0.0-xzq, dayplus: 1 not found, error: list index out of range


2025-08-29 01:30:40,965 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:41,599 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:41,637 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:41,644 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:41,820 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:42,362 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:42,407 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:42,415 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:42,590 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:43,182 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:43,223 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:43,230 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:43,410 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:43,950 - bytelinker - INFO - PostgreCore query success
2025-0

label: ifs, tag: 清鹏-lasso_lgbm_mlp-ec-V1.0.0-xzq, dayplus: 1 not found, error: list index out of range


2025-08-29 01:30:44,976 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:45,013 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:45,019 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:45,192 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:45,708 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:45,748 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:45,754 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:45,926 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:46,496 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:46,534 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:46,540 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:46,727 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:47,383 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:47,422 - bytelinker - INFO - PostgreCore disconnected


In [5]:
dbtype = "test"
date_begin = "2025-01-01 00:00:00"

from datetime import datetime, timedelta
import pytz

beijing = pytz.timezone("Asia/Shanghai")
today_bj = datetime.now(beijing).date()
date_end = (today_bj + timedelta(days=10)).strftime("%Y-%m-%d") + " 23:45:00"


for dayplus in range(4,15):
    #pv_pred = read_pred_pv_province(dayplus, dbtype, "清鹏-lgbm-V3.0.0-zxh-预测不限电发电量-单气象源ecmwfifs025", date_begin, date_end)
    #wind_pred = read_pred_wind_province(dayplus, dbtype, '清鹏-lgbm-V1.0.0-zhw-gfs', date_begin, date_end)
    load_pred = read_pred_load_province(dayplus, dbtype, '清鹏', date_begin, date_end)
    #pv_pred.to_parquet(f"../data/processed/pv/dayplus_{dayplus}_V3.parquet", engine='fastparquet')
    #wind_pred.to_parquet(f"../data/processed/wind/dayplus_{dayplus}_V1.parquet", engine='fastparquet')
    load_pred.to_parquet(f"data/processed/load/dayplus_{dayplus}_V0.parquet", engine='fastparquet')

2025-08-29 01:30:47,439 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:47,618 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:49,746 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:49,802 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:49,809 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:49,995 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:50,702 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:50,760 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:50,767 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:50,954 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:30:51,727 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:30:51,784 - bytelinker - INFO - PostgreCore disconnected
2025-08-29 01:30:51,791 - bytelinker - INFO - PostgreCore init
2025-08-29 01:30:51,971 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:3

In [6]:
wind_pred

,time,pred
0,2025-07-24 00:00:00,7188.380884
1,2025-07-24 00:15:00,7157.056759
2,2025-07-24 00:30:00,7374.807690
3,2025-07-24 00:45:00,7832.645256
4,2025-07-24 01:00:00,7968.953171
...,...,...
2971,2025-08-31 22:45:00,1269.555354
2972,2025-08-31 23:00:00,1375.294154
2973,2025-08-31 23:15:00,1375.294154
2974,2025-08-31 23:30:00,1375.294154


In [7]:
pv_pred.tail(100)

,time,pred
11900,2025-08-30 23:00:00,0.0
11901,2025-08-30 23:15:00,0.0
11902,2025-08-30 23:30:00,0.0
11903,2025-08-30 23:45:00,0.0
11904,2025-08-31 00:00:00,0.0
...,...,...
11995,2025-08-31 22:45:00,0.0
11996,2025-08-31 23:00:00,0.0
11997,2025-08-31 23:15:00,0.0
11998,2025-08-31 23:30:00,0.0


In [8]:
dfff=pd.read_parquet("data/processed/pv/dayplus_1_ifs.parquet", engine='fastparquet')

In [9]:
dfff

,time,pred
0,2025-08-28 00:00:00,0.0
1,2025-08-28 00:15:00,0.0
2,2025-08-28 00:30:00,0.0
3,2025-08-28 00:45:00,0.0
4,2025-08-28 01:00:00,0.0
...,...,...
11611,2025-08-27 22:45:00,0.0
11612,2025-08-27 23:00:00,0.0
11613,2025-08-27 23:15:00,0.0
11614,2025-08-27 23:30:00,0.0


In [10]:
dff1=pd.read_parquet("data/processed/load/dayplus_2_gfs.parquet", engine='fastparquet')
dff1.tail(10)

,time,pred
11510,2025-08-28 21:30:00,29671.284980
11511,2025-08-28 21:45:00,29383.376526
11512,2025-08-28 22:00:00,29312.034664
11513,2025-08-28 22:15:00,28850.893971
11514,2025-08-28 22:30:00,28446.601941
11515,2025-08-28 22:45:00,27972.723075
11516,2025-08-28 23:00:00,27957.852350
11517,2025-08-28 23:15:00,27580.626739
11518,2025-08-28 23:30:00,27114.088202
11519,2025-08-28 23:45:00,26988.941488


### 获取多日的价格预测

In [11]:
from datetime import datetime, timedelta
from pytsingrocbytelinker import PostgreLinker, DbType, WriteData, DBInfo
import numpy as np
import pandas as pd

def read_da_price(dayplus, dbtype, date_begin, date_end) -> None:
    byte_linker = PostgreLinker()
    if dbtype == 'test':
        byte_linker.connect(DbType.test)
    elif dbtype == 'prod':
        byte_linker.connect(DbType.prod)
    else:
        raise ValueError("dbtype must be 'test' or 'prod'")
    db_info = DBInfo()
    db_info.read_from_data(
        table_name="price_pred_day_ahead",
        key="price_pred_day_ahead",
        tags={ "points":96,"source": "清鹏", "dayplus": dayplus},
        daypoints=96,
        version="v1.0.0", # 1.1.0 
        column_name="pred",
        start_time=date_begin,
        end_time=date_end,
        region="cn-shanxi",
        value_tag="value",
    )
    data = byte_linker.query(db_info)
    byte_linker.disconnect()
    return data

date_begin = "2025-03-01"
date_end = "2025-07-31"
version=5

data = read_da_price(version,"test",f"{date_begin} 00:00:00",f"{date_end} 23:45:00")
data['time'] = pd.to_datetime(data['time']).dt.tz_localize(None)

## 注意，取出来的是一个分布，第0维表示中位数,1~100维表示顺序排序的100个采样

data_mid = data['pred'].values
data_mid = np.array([x[0] for x in data_mid])
data_mid_df = pd.DataFrame({
    "time":data['time'],
    "pred":data_mid
})

# data.to_csv(f"data/load_user/load_user_{date_str}.csv", index=False)

2025-08-29 01:30:59,915 - bytelinker - INFO - PostgreCore init
2025-08-29 01:31:00,409 - bytelinker - INFO - PostgreCore connected to test
2025-08-29 01:31:06,582 - bytelinker - INFO - PostgreCore query success
2025-08-29 01:31:06,630 - bytelinker - INFO - PostgreCore disconnected


In [12]:
import os, json

# —— 1) 叠成矩阵：shape = (N, 101)，第0列是中位数，1~100列是采样 ——
preds = np.vstack(data["pred"].to_numpy()).astype(float)

# 列名：p50 + s01..s100（便于下游读取）
cols = ["p50"] + [f"s{i:02d}" for i in range(1, preds.shape[1])]
dist_df = pd.DataFrame(preds, columns=cols)
dist_df.insert(0, "time", data["time"].to_numpy())

# —— 2) 输出目录与文件名 ——
out_dir = "./data/price_pred_da"
os.makedirs(out_dir, exist_ok=True)
date_tag = f"{date_begin}_to_{date_end}".replace("-", "")

# —— 3) 保存为 CSV / Parquet（两种都可留一份） ——
dist_df.to_csv(f"{out_dir}/da_dist_{version}.csv", index=False)
# 如果你更喜欢 Parquet（体积小、读取快），任选一个引擎：
# dist_df.to_parquet(f"{out_dir}/da_dist_{date_tag}.parquet", engine="pyarrow")
# dist_df.to_parquet(f"{out_dir}/da_dist_{date_tag}.parquet", engine="fastparquet")

In [13]:
dist_df.sort_values(by="time")

,time,p50,s01,s02,s03,s04,s05,s06,s07,s08,...,s91,s92,s93,s94,s95,s96,s97,s98,s99,s100
5735,2025-03-01 00:00:00,305.749420,258.895081,259.815735,268.845093,269.208069,276.440002,277.808014,277.811584,281.116272,...,336.962982,337.672272,338.240387,338.533356,339.929840,343.001648,360.281006,366.729614,367.498810,367.817352
5734,2025-03-01 00:15:00,314.599060,256.156036,284.866943,287.552826,287.578735,289.442017,290.872437,295.294708,296.038513,...,342.521332,345.733368,346.274506,349.088654,349.886200,353.343445,355.122131,358.529938,363.621155,366.620667
5733,2025-03-01 00:30:00,312.229980,277.035004,279.702789,282.704315,285.187775,288.548309,290.437927,291.678864,292.153412,...,339.425873,339.598297,340.761444,344.094238,344.303650,345.298401,349.211090,350.376099,351.515106,371.539490
5732,2025-03-01 00:45:00,309.552307,270.143921,278.874847,278.940979,281.453186,281.907104,285.427612,286.425354,287.570587,...,337.500763,339.224365,339.936249,340.245392,341.459534,344.472504,345.011841,354.759308,355.036560,363.333191
5731,2025-03-01 01:00:00,305.897003,263.901764,267.500488,274.486023,278.889404,280.123901,280.184601,280.845154,282.733978,...,334.667664,339.199493,340.259369,341.503052,342.726166,343.200806,346.538269,347.154602,350.194611,364.284363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14393,2025-07-31 22:45:00,355.187439,270.101562,279.776031,284.270416,289.217804,297.348450,298.063629,303.817444,304.057465,...,578.381104,634.125488,736.500732,744.087036,824.022766,832.139404,927.194275,1101.819702,1204.176025,1254.744141
14394,2025-07-31 23:00:00,344.001587,266.101440,274.077148,275.845306,282.289795,287.285004,289.769531,295.178528,295.987823,...,396.759460,400.544678,489.888519,534.288757,549.442566,621.676025,712.216003,883.473755,1061.824585,1211.410400
14395,2025-07-31 23:15:00,338.851440,267.903168,277.640472,277.839264,287.283722,288.744659,290.181305,296.067108,300.636810,...,389.400848,389.790863,393.238373,407.462219,566.827087,682.583496,725.238281,729.985352,1012.774902,1083.441406
14396,2025-07-31 23:30:00,335.882843,273.377136,277.774750,280.336365,283.370911,284.831390,288.523407,292.608704,295.957428,...,381.461853,384.090332,388.172791,388.627197,392.336273,397.325409,678.312500,721.741150,751.080200,1121.028687
